In [50]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],
    license_uuid="default",
)

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


In [ ]:
%%writefile wsyz_projekt.mod

set PRODUCENCI;
set MAGAZYNY;
set SKLEPY;
set WARZYWA;
param HORYZONT;
set TYGODNIE := 1..HORYZONT;

param KosztTransportuTonakm;

param ProdukcjaMax{PRODUCENCI, WARZYWA} >= 0;
param PojemnoscMagazynuCentralnego{MAGAZYNY} >= 0;
param OdlegloscPM{PRODUCENCI, MAGAZYNY} >= 0;

param SprzedazBazowa{SKLEPY, WARZYWA} >= 0;

# Definicja sezonowości
param MnoznikSezonowy{t in TYGODNIE} :=
    if t <= 13 then 1.3
    else if t <= 26 then 0.8
    else if t <= 39 then 0.6
    else 1.1;

# Sprzedaż prognozowana
param SprzedazPrognozowana{s in SKLEPY, w in WARZYWA, t in TYGODNIE}
    := SprzedazBazowa[s,w] * MnoznikSezonowy[t];

# Średnia tygodniowa
param SredniaSprzedaz{s in SKLEPY, w in WARZYWA}
    := (sum{t in TYGODNIE} SprzedazPrognozowana[s,w,t]) / HORYZONT;

# Limity magazynowe
param PojemnoscSklepu{s in SKLEPY, w in WARZYWA} := 2 * SredniaSprzedaz[s,w];
param ZapasMin{s in SKLEPY, w in WARZYWA} := 0.1 * SredniaSprzedaz[s,w];

param OdlegloscMS{MAGAZYNY, SKLEPY} >= 0;

var x_pm{PRODUCENCI, MAGAZYNY, WARZYWA} >= 0;
var x_ms{MAGAZYNY, SKLEPY, WARZYWA, TYGODNIE} >= 0;
var zapas_s{SKLEPY, WARZYWA, 0..HORYZONT} >= 0;

minimize KosztCalkowity:
    sum{p in PRODUCENCI, m in MAGAZYNY, w in WARZYWA}
        x_pm[p,m,w] * OdlegloscPM[p,m] * KosztTransportuTonakm +
    sum{m in MAGAZYNY, s in SKLEPY, w in WARZYWA, t in TYGODNIE}
        x_ms[m,s,w,t] * OdlegloscMS[m,s] * KosztTransportuTonakm;

subject to LimitProdukcji {p in PRODUCENCI, w in WARZYWA}:
    sum{m in MAGAZYNY} x_pm[p,m,w] <= ProdukcjaMax[p,w];

subject to LimitPojemnosciMC {m in MAGAZYNY}:
    sum{p in PRODUCENCI, w in WARZYWA} x_pm[p,m,w] <= PojemnoscMagazynuCentralnego[m];

subject to BilansMagazynuCentralnego {m in MAGAZYNY, w in WARZYWA}:
    sum{p in PRODUCENCI} x_pm[p,m,w] = sum{s in SKLEPY, t in TYGODNIE} x_ms[m,s,w,t];

subject to BilansSklepu {s in SKLEPY, w in WARZYWA, t in TYGODNIE}:
    zapas_s[s,w,t] = zapas_s[s,w,t-1] + sum{m in MAGAZYNY} x_ms[m,s,w,t] - SprzedazPrognozowana[s,w,t];

subject to PojemnoscMagazynuSklepu {s in SKLEPY, w in WARZYWA, t in TYGODNIE}:
    zapas_s[s,w,t] <= PojemnoscSklepu[s,w];

subject to MinimalnyZapasSklepu {s in SKLEPY, w in WARZYWA, t in TYGODNIE}:
    zapas_s[s,w,t] >= ZapasMin[s,w];

subject to ZapasPoczatkowy {s in SKLEPY, w in WARZYWA}:
    zapas_s[s,w,0] = ZapasMin[s,w];

Overwriting wsyz_projekt.mod


In [52]:
%%writefile wsyz_projekt.dat

set PRODUCENCI :=
  P1  # Błonie
  P2  # Książenice
  P3  # Góra Kalwaria
  P4  # Otwock
  P5  # Wołomin
  P6; # Legionowo

set MAGAZYNY :=
  M1  # Pruszków
  M2  # Piaseczno
  M3; # Zielonka

set WARZYWA :=
  Ziemniaki
  Kapusta
  Buraki
  Marchew;

set SKLEPY :=
  S1    # Erazma Ciołka 25, 01-445 Warszawa
  S2    # Wolska 64, 01-134 Warszawa
  S3    # Gościnna 37, Blizne Łaszczyńskiego
  S4    # Pirenejska 2c, 01-493 Warszawa
  S5    # Zygmunta Modzelewskiego 54B, 02-679 Warszawa
  S6    # Puławska 69, 02-595 Warszawa
  S7    # Miedziana 3a/1c, 00-814 Warszawa
  S8    # Dobra 54, 00-341 Warszawa
  S9    # Wrzesińska 4/Lok. U2 i U3, 03-713 Warszawa
  S10;  # Żupnicza 23, 03-821 Warszawa

param HORYZONT := 52;
param KosztTransportuTonakm := 5;

param ProdukcjaMax: Ziemniaki Kapusta Buraki Marchew :=
                P1  160       180     120    160
                P2  90        100     140    80
                P3  160       70      190    100
                P4  260       50      60     100
                P5  200       230     190    110
                P6  250       130     80     100;

param PojemnoscMagazynuCentralnego :=
                M1 800
                M2 1200
                M3 750;

# Na podstawie drogi proponowanej przez Google Maps
param OdlegloscPM:  M1         M2          M3 :=
                P1  17         39          44
                P2  18         29          54
                P3  42         20          44
                P4  37         28          25
                P5  42         38          9
                P6  38         43          25;

param OdlegloscMS: S1 S2 S3 S4 S5 S6 S7 S8 S9 S10 :=
                M1 16 15 14 15 15 18 15 22 23 22
                M2 26 24 26 29 12 14 17 21 22 24
                M3 23 23 25 22 20 18 16 20 12 14;

# Bazowa sprzedaż - każdy sklep ma inne realne zapotrzebowanie
param SprzedazBazowa: Ziemniaki Kapusta Buraki Marchew :=
  S1  1.2  0.7  0.4  0.8
  S2  1.0  0.5  0.3  0.6
  S3  0.5  0.3  0.2  0.4
  S4  0.9  0.6  0.4  0.7
  S5  2.0  1.2  0.8  1.5
  S6  1.8  1.0  0.7  1.3
  S7  0.8  0.4  0.3  0.5
  S8  0.7  0.4  0.2  0.5
  S9  1.1  0.6  0.4  0.8
  S10 1.6  0.9  0.6  1.2;

Overwriting wsyz_projekt.dat


In [54]:
%%ampl_eval
reset;
model wsyz_projekt.mod;
data wsyz_projekt.dat;
option solver highs;
solve;

display KosztCalkowity;

# Transport roczny od producentów do magazynów
print "--- DECYZJE A: TRANSPORT ROCZNY P -> M ---";
display x_pm;

# Transport tygodniowy z Magazynów do Sklepów na przykładzie 1 tygodnia
print "--- DECYZJE B: TRANSPORT TYGODNIOWY M -> S (TYDZIEN 1) ---";
display {m in MAGAZYNY, s in SKLEPY, w in WARZYWA} x_ms[m, s, w, 1];

# Stan zapasów w sklepach na koniec 1 tygodnia
print "--- DECYZJE C: STAN ZAPASÓW W SKLEPACH (KONIEC TYGODNIA 1) ---";
display {s in SKLEPY, w in WARZYWA} zapas_s[s, w, 1];


HiGHS 1.11.0: HiGHS 1.11.0: optimal solution; objective 221613.5
4124 simplex iterations
0 barrier iterations

"option abs_boundtol 2.220446049250313e-16;"
or "option rel_boundtol 1.947759692324836e-16;"
will change deduced dual values.

KosztCalkowity = 221614

--- DECYZJE A: TRANSPORT ROCZNY P -> M ---
x_pm [*,M1,*]
:   Buraki Kapusta  Marchew Ziemniaki    :=
P1   22.42   96.04   160      160
P2    0       0       40.02    53.04
P3    0       0        0        0
P4    0       0        0        0
P5    0       0        0        0
P6    0       0        0        0

 [*,M2,*]
:  Buraki Kapusta Marchew Ziemniaki    :=
P1    0       0        0        0
P2    0       0        0        0
P3    0       0      100      160
P4    0       0        0        0
P5    0       0        0        0
P6    0       0        0        0

 [*,M3,*]
:  Buraki Kapusta Marchew Ziemniaki    :=
P1     0       0       0        0
P2     0       0       0        0
P3     0       0       0        0
P4 